<a href="https://colab.research.google.com/github/FaiqAminuddin/ml/blob/main/TugasAkhirFaiq_ML_klasifikasi_Gambar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Klasifikasi Gambar: Animals
- **Nama:** Faiq Aminuddin
- **Email:** faiq.aminuddin.demak@gmail.com
- **ID Dicoding:** faiqaminuddin

## Import Semua Packages/Library yang Digunakan

In [1]:
# Import library yang diperlukan
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import zipfile
import os




## Data Preparation

### Data Loading

In [2]:
"""
versi 1
! pip install kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
"""
#menghubungkan dengan Kaggle versi 2 (pakai key dan user name)
!pip install -q kaggle
os.environ["KAGGLE_KEY"] ='cccdae8f025910c3006efc8880f868d0'
os.environ["KAAGLE_USERNAME"] = 'faiqaminuddin'


In [3]:
# Download dataset
! kaggle datasets download -d borhanitrash/animal-image-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/borhanitrash/animal-image-classification-dataset
License(s): MIT
 73% 28.0M/38.4M [00:00<00:00, 87.8MB/s]
100% 38.4M/38.4M [00:00<00:00, 85.8MB/s]


In [4]:
# Ekstrak file zip
with zipfile.ZipFile('animal-image-classification-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('animals')



In [5]:
# Path untuk dataset
base_dir = 'animals/Animals'
# Cek struktur direktori untuk memastikan path yang benar
print(os.listdir(base_dir))

['snakes', 'cats', 'dogs']


### Data Preprocessing

#### Split Dataset

In [6]:
# ImageDataGenerator untuk augmentasi dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2 # Set validation split to 20%
)



In [7]:
# Data generator untuk training dan validation set
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training' # Set sebagai training data
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation' # Set sebagai validation data
)



Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


## Modelling

In [8]:
# Membuat model Sequential
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax') # 3 kelas: kucing, ular, anjing
])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])



In [ ]:
# Melatih model
model.fit(
      train_generator,
      steps_per_epoch=25,  # 25 batch  pada setiap epoch
      epochs=50, # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # 5 batch dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


## Evaluasi dan Visualisasi

In [ ]:
# Evaluasi model
scores = model.evaluate(validation_generator, verbose=1)
print(f"Validation Accuracy: {scores[1]*100:.2f}%")

# Akurasi minimum 85%
if scores[1] >= 0.85:
    print("Model memenuhi kriteria dengan akurasi di atas 85%.")
else:
    print("Model belum memenuhi kriteria dengan akurasi di bawah 85%.")

In [ ]:
# Evaluate model
train_loss, train_acc = model.evaluate(x_train, y_train, verbose=2)
val_loss, val_acc = model.evaluate(x_val, y_val, verbose=2)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print(f'Training Accuracy: {train_acc}')
print(f'Validation Accuracy: {val_acc}')
print(f'Test Accuracy: {test_acc}')

In [ ]:
# Plot accuracy and loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()



## Konversi Model

In [ ]:
# Save model in SavedModel format
model.save('model_saved/')

# Convert model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Convert model to TensorFlow.js format
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'model_tfjs/')

## Inference (Optional)

In [ ]:
# Import library yang diperlukan
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.preprocessing import image

# Fungsi untuk memproses dan memprediksi gambar yang diunggah
def upload_and_predict(model):
    # Unggah gambar
    uploaded = files.upload()
    for fn in uploaded.keys():
        # Load dan proses gambar
        img_path = fn
        img = image.load_img(img_path, target_size=(150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalisasi

        # Prediksi gambar
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions[0])

        # Mapping kelas
        class_labels = ['cats', 'dogs', 'snakes']

        # Tampilkan hasil prediksi
        print(f"Prediction for {fn}: {class_labels[predicted_class]}")
        plt.imshow(img)
        plt.title(f"Prediction: {class_labels[predicted_class]}")
        plt.axis('off')
        plt.show()

# Panggil fungsi untuk mengunggah dan memprediksi gambar
upload_and_predict(model)

In [ ]:
 pip freeze requirements.txt